In [30]:
import pandas as pd, numpy as np

In [31]:
!pip install trueskill

In [32]:
from trueskill import TrueSkill, Rating, rate_1vs1

In [60]:
ts = TrueSkill(draw_probability=0.01)
beta = 25/6

In [61]:
def win_proba(t1, t2):
    delta_mu = t1.mu - t2.mu
    sum_sigma = (t1.sigma * t1.sigma) + (t2.sigma * t2.sigma)
    denom = np.sqrt(2 * (beta*beta) + (sum_sigma))
    return ts.cdf(delta_mu/denom)

In [62]:
submission = pd.read_csv('./SampleSubmissionStage1.csv')
submission[['Season', 'Team1', 'Team2']] = submission.apply(lambda x: pd.Series([int(t) for t in x.ID.split('_')]), axis=1)


In [63]:
tourney = pd.read_csv('./DataFiles/RegularSeasonCompactResults.csv')
team_ids = np.unique(np.concatenate([tourney.WTeamID.values, tourney.LTeamID.values]))
ratings = {tid: ts.Rating() for tid in team_ids}

In [64]:
def feed_season_results(season):
    print('Season = {}'.format(season))
    tourney_1 = tourney[tourney.Season == season]
    for x in tourney_1.itertuples():
        ratings[x.WTeamID], ratings[x.LTeamID] = rate_1vs1(ratings[x.WTeamID], ratings[x.LTeamID])

In [65]:
def update_pred(season):
    beta = np.std([x.mu for x in ratings.values()])
    print('Beta = {}'.format(beta))
    submission.loc[submission.Season==season, 'Pred'] = submission[submission.Season==season].apply(lambda x:win_proba(ratings[x.Team1], ratings[x.Team2]), axis=1)

In [66]:
for season in sorted(tourney.Season.unique())[:-4]:
    feed_season_results(season)
    
update_pred(2014)
feed_season_results(2014)
update_pred(2015)
feed_season_results(2015)
update_pred(2016)
feed_season_results(2016)
update_pred(2017)
feed_season_results(2017)

submission.drop(['Season', 'Team1', 'Team2'], axis=1, inplace=True)
submission.to_csv('trueskill_estimator_1.csv', index=None)

Season = 1985
Season = 1986
Season = 1987
Season = 1988
Season = 1989
Season = 1990
Season = 1991
Season = 1992
Season = 1993
Season = 1994
Season = 1995
Season = 1996
Season = 1997
Season = 1998
Season = 1999
Season = 2000
Season = 2001
Season = 2002
Season = 2003
Season = 2004
Season = 2005
Season = 2006
Season = 2007
Season = 2008
Season = 2009
Season = 2010
Season = 2011
Season = 2012
Season = 2013
Beta = 5.173598443091855
Season = 2014
Beta = 5.2203241593615415
Season = 2015
Beta = 5.175775424826821
Season = 2016
Beta = 5.158745585314826
Season = 2017
